# Creating and Managing Indexes
Explanation:
- Indexes are used to improve the performance of queries by allowing the database to quickly locate the rows that match certain criteria.
- The `CREATE INDEX` statement is used to create an index on one or more columns of a table.
- The `CREATE UNIQUE INDEX` statement creates a unique index, which ensures that the indexed columns contain only unique values.
- Indexes can be created on a single column or multiple columns.
- Constraints are used to enforce rules and relationships on the data in a table.
- The `ALTER TABLE` statement is used to add constraints to an existing table.
- The `PRIMARY KEY` constraint defines a column or a set of columns as the primary key for a table.
- The `UNIQUE` constraint ensures that the values in the specified column(s) are unique.
- The `FOREIGN KEY` constraint establishes a relationship between two tables by referencing the primary key of another table.
- Indexes and constraints can be dropped using the `DROP INDEX` and `ALTER TABLE DROP CONSTRAINT` statements, respectively.

In [ ]:
-- Creating a table
CREATE TABLE employees (
    id INT PRIMARY KEY,
    name VARCHAR(100),
    age INT,
    salary DECIMAL(10, 2)
);

-- Creating an index on a single column
CREATE INDEX idx_name ON employees (name);

-- Creating a unique index on a single column
CREATE UNIQUE INDEX idx_id ON employees (id);

-- Creating an index on multiple columns
CREATE INDEX idx_age_salary ON employees (age, salary);

-- Creating a unique index on multiple columns
CREATE UNIQUE INDEX idx_name_age ON employees (name, age);

-- Creating a primary key constraint
ALTER TABLE employees ADD CONSTRAINT pk_id PRIMARY KEY (id);

-- Creating a unique constraint
ALTER TABLE employees ADD CONSTRAINT uc_name UNIQUE (name);

-- Creating a foreign key constraint
CREATE TABLE departments (
    id INT PRIMARY KEY,
    name VARCHAR(100)
);

ALTER TABLE employees ADD CONSTRAINT fk_department_id FOREIGN KEY (department_id) REFERENCES departments(id);

-- Dropping an index
DROP INDEX idx_name;

-- Dropping a constraint
ALTER TABLE employees DROP CONSTRAINT pk_id;

# Understanding Primary Keys
Explanation:
In this code snippet, we demonstrate the usage of primary keys in SQL. 

First, we create a table called `employees` with an `id` column as the primary key. We insert some data into the table and attempt to insert a duplicate primary key value, which results in a primary key violation error.

Next, we show how to retrieve data from the table using a simple `SELECT` statement. We also demonstrate updating and deleting records using the primary key.

Then, we add a primary key constraint to an existing table using the `ALTER TABLE` statement. We attempt to insert a duplicate primary key value again, which results in an error. We then drop the primary key constraint and successfully insert the record with the previously duplicate primary key value.

Finally, we create another table called `products` with an auto-increment primary key column using the `SERIAL` data type. We insert data into the table without specifying the primary key value, and retrieve the data using a `SELECT` statement. We also attempt to insert a record with a specific primary key value, which results in an error due to the auto-increment property of the primary key column.

Note: The syntax and behavior of primary keys may vary slightly depending on the specific SQL database system being used.

In [ ]:
-- Create a table with a primary key
CREATE TABLE employees (
    id INT PRIMARY KEY,
    name VARCHAR(50),
    age INT
);

-- Insert some data into the table
INSERT INTO employees (id, name, age) VALUES (1, 'John Doe', 30);
INSERT INTO employees (id, name, age) VALUES (2, 'Jane Smith', 25);
INSERT INTO employees (id, name, age) VALUES (3, 'Mike Johnson', 35);

-- Attempt to insert a duplicate primary key value
INSERT INTO employees (id, name, age) VALUES (1, 'Duplicate', 40);
-- This will result in a primary key violation error

-- Retrieve data from the table
SELECT * FROM employees;
-- Expects to print:
-- id |    name     | age
-- ----+-------------+-----
--  1  | John Doe    |  30
--  2  | Jane Smith  |  25
--  3  | Mike Johnson|  35

-- Update a record using the primary key
UPDATE employees SET age = 31 WHERE id = 1;

-- Delete a record using the primary key
DELETE FROM employees WHERE id = 2;

-- Attempt to update or delete a non-existing record
UPDATE employees SET age = 32 WHERE id = 4;
-- This will not update any records

DELETE FROM employees WHERE id = 4;
-- This will not delete any records

-- Add a primary key constraint to an existing table
ALTER TABLE employees ADD CONSTRAINT pk_employees PRIMARY KEY (id);

-- Attempt to insert a duplicate primary key value after adding the constraint
INSERT INTO employees (id, name, age) VALUES (1, 'Duplicate', 40);
-- This will result in a primary key violation error

-- Drop the primary key constraint
ALTER TABLE employees DROP CONSTRAINT pk_employees;

-- Attempt to insert a duplicate primary key value after dropping the constraint
INSERT INTO employees (id, name, age) VALUES (1, 'Duplicate', 40);
-- This will insert the record successfully

-- Add an auto-increment primary key column
CREATE TABLE products (
    id SERIAL PRIMARY KEY,
    name VARCHAR(50),
    price DECIMAL(10, 2)
);

-- Insert data into the table without specifying the primary key value
INSERT INTO products (name, price) VALUES ('Product 1', 10.99);
INSERT INTO products (name, price) VALUES ('Product 2', 19.99);

-- Retrieve data from the table
SELECT * FROM products;
-- Expects to print:
-- id |   name    | price
-- ----+-----------+-------
--  1  | Product 1 | 10.99
--  2  | Product 2 | 19.99

-- Attempt to insert a record with a specific primary key value
INSERT INTO products (id, name, price) VALUES (1, 'Duplicate', 40);
-- This will result in an error since the primary key is auto-incremented

-- Drop the table
DROP TABLE employees;
DROP TABLE products;

# Foreign Keys and Relations
Explanation:
In this code snippet, we demonstrate the usage of foreign keys and relations in SQL. 

First, we create two tables: "Customers" and "Orders". The "Customers" table has a primary key column `customer_id` and a `customer_name` column. The "Orders" table also has a primary key column `order_id`, an `order_date` column, and a `customer_id` column.

To establish a relationship between the two tables, we define a foreign key constraint on the `customer_id` column in the "Orders" table. The foreign key references the `customer_id` column in the "Customers" table, ensuring that only valid customer IDs can be inserted into the "Orders" table.

We then insert sample data into both tables to demonstrate the relationship. Two customers are inserted into the "Customers" table, and two orders are inserted into the "Orders" table, each associated with a specific customer.

Finally, we perform a query to retrieve the orders along with the corresponding customer information. The `JOIN` keyword is used to combine the "Orders" and "Customers" tables based on the matching `customer_id` values. The result includes the order ID, order date, and customer name.

Expected output:
```
order_id | order_date  | customer_name
--------------------------------------
1        | 2022-01-01 | John Doe
2        | 2022-01-02 | Jane Smith
```

In [ ]:
-- Create a table for the "Customers" entity
CREATE TABLE Customers (
    customer_id INT PRIMARY KEY,
    customer_name VARCHAR(50)
);

-- Create a table for the "Orders" entity
CREATE TABLE Orders (
    order_id INT PRIMARY KEY,
    order_date DATE,
    customer_id INT,
    FOREIGN KEY (customer_id) REFERENCES Customers(customer_id)
);

-- Insert sample data into the "Customers" table
INSERT INTO Customers (customer_id, customer_name)
VALUES (1, 'John Doe'),
       (2, 'Jane Smith');

-- Insert sample data into the "Orders" table
INSERT INTO Orders (order_id, order_date, customer_id)
VALUES (1, '2022-01-01', 1),
       (2, '2022-01-02', 2);

-- Query to retrieve orders along with customer information
SELECT o.order_id, o.order_date, c.customer_name
FROM Orders o
JOIN Customers c ON o.customer_id = c.customer_id;

# Constraints and Enforcement
Explanation:
- The code snippet demonstrates the usage of constraints in SQL, including primary key, check, unique, and foreign key constraints.
- The `employees` table has a primary key constraint on the `id` column, a check constraint on the `age` column to enforce a minimum age of 18, and a foreign key constraint on the `department_id` column referencing the `departments` table.
- The `orders` table demonstrates the usage of a composite primary key constraint on the `order_id` and `product_id` columns.
- The `customers` table has a unique constraint on the `email` column to ensure uniqueness of email addresses.
- The `departments` table serves as the referenced table for the foreign key constraint in the `employees` table.
- The code includes insert statements that violate some of the constraints to demonstrate their enforcement.
- Finally, the code performs select queries on all the tables to show the data and constraints in action.

Expected output:
- The `employees` table should have two rows (John Doe and Jane Smith) since the insertion of Bob Johnson violates the check constraint.
- The `orders` table should have three rows with different combinations of `order_id` and `product_id`.
- The `customers` table should have two rows (John Doe and Jane Smith) since the insertion of John Doe violates the unique constraint.
- The `departments` table should have two rows (Sales and Marketing).
- The select queries should display the data from the respective tables.

In [ ]:
-- Create a table with constraints
CREATE TABLE employees (
    id INT PRIMARY KEY,
    name VARCHAR(50) NOT NULL,
    age INT CHECK (age >= 18),
    department_id INT,
    CONSTRAINT fk_department FOREIGN KEY (department_id) REFERENCES departments(id)
);

-- Insert data into the table
INSERT INTO employees (id, name, age, department_id) VALUES (1, 'John Doe', 25, 1);
INSERT INTO employees (id, name, age, department_id) VALUES (2, 'Jane Smith', 30, 2);
INSERT INTO employees (id, name, age, department_id) VALUES (3, 'Bob Johnson', 17, 1); -- This should violate the CHECK constraint

-- Create a table with composite primary key
CREATE TABLE orders (
    order_id INT,
    product_id INT,
    quantity INT,
    PRIMARY KEY (order_id, product_id)
);

-- Insert data into the table with composite primary key
INSERT INTO orders (order_id, product_id, quantity) VALUES (1, 100, 5);
INSERT INTO orders (order_id, product_id, quantity) VALUES (1, 101, 3);
INSERT INTO orders (order_id, product_id, quantity) VALUES (2, 100, 2);

-- Create a table with unique constraint
CREATE TABLE customers (
    customer_id INT,
    email VARCHAR(50) UNIQUE
);

-- Insert data into the table with unique constraint
INSERT INTO customers (customer_id, email) VALUES (1, 'john.doe@example.com');
INSERT INTO customers (customer_id, email) VALUES (2, 'jane.smith@example.com');
INSERT INTO customers (customer_id, email) VALUES (3, 'john.doe@example.com'); -- This should violate the unique constraint

-- Create a table with foreign key constraint
CREATE TABLE departments (
    id INT PRIMARY KEY,
    name VARCHAR(50)
);

-- Insert data into the table with foreign key constraint
INSERT INTO departments (id, name) VALUES (1, 'Sales');
INSERT INTO departments (id, name) VALUES (2, 'Marketing');

-- Attempt to insert data into employees table with invalid foreign key
INSERT INTO employees (id, name, age, department_id) VALUES (4, 'Alice Johnson', 28, 3); -- This should violate the foreign key constraint

-- Query the employees table
SELECT * FROM employees;

-- Query the orders table
SELECT * FROM orders;

-- Query the customers table
SELECT * FROM customers;

-- Query the departments table
SELECT * FROM departments;